In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
############################################################
'''
Part 1: RK4 Validation with simple spring-damper model + PID control
'''
############################################################

## parameter setup
# Integration parameter 
h=1/100  # step size    (sec)
t0=0    # initial time (sec)
tf=10    # final time (sec)


time=np.linspace(t0,tf,int((tf-t0)/h))

## motion initial state
x_t0=1.0
v_t0=0
a_t0=0

x_target=3.0         # height target [m]

motion_t0=np.array([x_t0,v_t0])
motion_t=np.zeros((len(motion_t0),len(time)))


## PID parameters
kp=0.5
ki=1.0
kd=1.5

integral=0   # initialize the integral
lasterr=0    # for the kd  

In [ ]:
# Butcher Array
RK4matrix=np.array([
      [0,0,0,0],
      [1/4,1/4,0,0],
      [27/40,-189/800,729/800,0],       
      [1,214/891,1/33,650/891],       
      [214/891,1/33,650/891,0],      
   ])


def rk4_ex(f,t,x,h,*args):  #*args allows unknown number of arguments
   k1=f(t+h*RK4matrix[0][0],x+h*RK4matrix[0][1],*args)
   k2=f(t+h*RK4matrix[1][0],x+h*k1*RK4matrix[1][1],*args)
   k3=f(t+h*RK4matrix[2][0],x+h*k2*RK4matrix[2][1],*args)
   k4=f(t+h*RK4matrix[3][0],x+h*k3*RK4matrix[3][1],*args)

   xnext=x+h*(k1*RK4matrix[4][0]+k2*RK4matrix[4][1]+k3*RK4matrix[4][2]+k4*RK4matrix[4][3])
   return xnext



def dx(t,motion,dotm,ve,*args):
   #xdot=motion[1]
   #xdotdot=5
   
   xdot=motion[1]
   xdotdot=-3.0*(motion[0]-x_target)-1.5*motion[1]+dotm
   return np.array([xdot,xdotdot])

def PID(x,h,kp,ki,kd):
   global integral, lasterr

   err=x-x_target
   integral=integral+ki*err*h
   derr=(err-lasterr)/h
   
   lasterr=err

   return -(kp*err+kd*derr+ki*integral)


def stateupdate(t,h,motion):
   
   x=motion[0]

   md4n=PID(x,h,kp,ki,kd)
   ve=2.5
   motion_n=rk4_ex(dx,t,motion,h,md4n,ve)


   return motion_n

In [ ]:
# Main iteration loop
motion_ti=motion_t0


for ti in range(len(time)-1):
    #
    t=time[ti]



    # calculte the new value (RK4/Algebraic update)
    motion_ti=stateupdate(t,h,motion_ti)

    #store the step
    motion_t[:,ti]=motion_ti

    #print(f'Step {t} of {len(time)-1}'.format())


print("last position is", motion_t[0,-2])

In [ ]:
# Visualization

# part 3: motion
plt.figure(figsize=(24,5))
plt.subplot(121)
plt.plot(time[:-1], motion_t[0, :-1], label='x')
plt.xlabel('Time [s]')
plt.ylabel('Height [m]')
plt.grid()


plt.subplot(122)
plt.plot(time[:-1], motion_t[1, :-1], label='v')
plt.xlabel('Time [s]')
plt.ylabel('Velocity [m/s]')
plt.grid()

In [ ]:
############################################################
'''
Part 2: RK4 Validation with rocket dynamic + PID control
'''
# Now we introduce the rocket dynamic here
############################################################

## parameter setup
# Integration parameter 
h=1/1000  # step size    (sec)
t0=0    # initial time (sec)
tf=10    # final time (sec)


time=np.linspace(t0,tf,int((tf-t0)/h))

## motion initial state
x_t0=0.0
v_t0=0
a_t0=0

x_target=4.0         # height target [m]

motion_t0=np.array([x_t0,v_t0])
motion_t=np.zeros((len(motion_t0),len(time)))

## mass
m_hop=3.5    # mass of hopper [kg]
m_hose=1     # mass of hose   [kg]
k_hose=6     # variable hopper mass [N/m]
F_RR=10      # rolling resistence [N]

## constant
R=8314            # universal gas constant [J/kmol*K]
M=28.013           # nitrogen gas  [kg/kmol]
gamma=1.4          # ratio of specific heats (diatomic gas)
g=9.81             # [m/s^2]

## Nozzle
Th_d=9e-3                        # throat diameter [m]
Th_exit_d=11e-3                  # exit diameter [m]
A_throat=np.pi*(Th_d/2)**2      #[m^2]
A_e=np.pi*(Th_exit_d/2)**2      #[m^2]
epsilon=A_e/A_throat            #expansion ratio
T_4=273                         # nozzle inlet temperature [k]


## Mach number
Me=1.8486328125000004        # from the main file, no longer do iteration

## PID parameters (gain has to be larger since the step size is really small)
kp=45
ki=10
kd=33

integral=0   # initialize the integral
lasterr=0    # for the kd  

prev_pose=x_t0


md4n_term=A_throat / np.sqrt(T_4*R/(M*gamma)) * np.power((gamma+1)/2, -(gamma+1)/(2*(gamma-1)))
print(md4n_term)
print(A_throat)
print(np.sqrt(T_4*R/(M*gamma)))
print(np.power((gamma+1)/2, -(gamma+1)/(2*(gamma-1))))

In [ ]:
# Butcher Array
RK4matrix=np.array([
      [0,0,0,0],
      [1/4,1/4,0,0],
      [27/40,-189/800,729/800,0],       
      [1,214/891,1/33,650/891],       
      [214/891,1/33,650/891,0],      
   ])


def rk4_ex(f,t,x,h,*args):  #*args allows unknown number of arguments
   k1=f(t+h*RK4matrix[0][0],x+h*RK4matrix[0][1],*args)
   k2=f(t+h*RK4matrix[1][0],x+h*k1*RK4matrix[1][1],*args)
   k3=f(t+h*RK4matrix[2][0],x+h*k2*RK4matrix[2][1],*args)
   k4=f(t+h*RK4matrix[3][0],x+h*k3*RK4matrix[3][1],*args)

   xnext=x+h*(k1*RK4matrix[4][0]+k2*RK4matrix[4][1]+k3*RK4matrix[4][2]+k4*RK4matrix[4][3])
   return xnext



def dx(t,motion,dotm,ve,*args):
   xdot=motion[1]
   xdotdot=dotm*ve/m_hop-g-F_RR/m_hop*np.sign(motion[1])-k_hose/m_hose*xdot*h   # the last term (hose) assumes that the hose become the damping term

   return np.array([xdot,xdotdot])

def PID(x,h,kp,ki,kd):
   global integral, lasterr
  
   err=x-x_target
   integral=integral+ki*err*h
   derr=(err-lasterr)/h
   
   lasterr=err

   PIDlaw=-(kp*err+kd*derr+ki*integral)
   #print("error: ",err, "integral: ", integral, "derror: ",derr)

   if PIDlaw<1:
      output=1
   elif PIDlaw>11:
      output=11
   else:
      output=PIDlaw

   return output


def stateupdate(t,h,motion):
   
   x=motion[0]
   
   p4n=PID(x,h,kp,ki,kd)*1e5                                                       # control input (modify once deploy the controller)
   md4n=A_throat*p4n/(np.sqrt(T_4*R/(M*gamma)))*np.power((gamma+1)/2,-(gamma+1)/(2*(gamma-1)))

   # Nozzle exit
   pe=p4n/np.power(1+((gamma-1)/2*Me**2), gamma/(gamma-1))                         #exit pressure (use the assumption of p_4/p_ambient>1.8)
   ve=np.sqrt((2*gamma*R*T_4)/(gamma-1)/M*(1-np.power(pe/p4n,(gamma-1)/gamma)))    #exit velocity

   #print("ve",ve)

   print("ve: ",ve,"pe: ",pe," p4: ",p4n, "md4n", md4n)

   # update the motion
   motion_n=rk4_ex(dx,t,motion,h,md4n,ve,prev_pose)

   print("x: ",motion_n[0])

   return motion_n


md4n_term=A_throat / np.sqrt(T_4*R/(M*gamma)) * np.power((gamma+1)/2, -(gamma+1)/(2*(gamma-1)))
print(md4n_term)

In [ ]:
# Main iteration loop
motion_ti=motion_t0


for ti in range(len(time)):
    #
    t=time[ti]

    # calculte the new value (RK4/Algebraic update)
    motion_ti=stateupdate(t,h,motion_ti)

    #store the step
    motion_t[:,ti]=motion_ti
    
    prev_pose=motion_t[0,ti-1] if ti>0 else x_t0
   
    #print(f'Step {t} of {len(time)-1}'.format())


print("last position is", motion_t[0,-1])

In [ ]:
# Visualization

# part 3: motion
plt.figure(figsize=(24,5))
plt.subplot(121)
plt.plot(time[:-1], motion_t[0, :-1], label='x')
plt.xlabel('Time [s]')
plt.ylabel('Height [m]')
plt.grid()


plt.subplot(122)
plt.plot(time[:-1], motion_t[1, :-1], label='v')
plt.xlabel('Time [s]')
plt.ylabel('Velocity [m/s]')
plt.grid()

In [ ]:
############################################################
'''
Side Note: discuss the nozzle mass flow rate (chocked) (dot_m) / exhaust velocity (ve) respect to nozzle inlet pressure 
'''

# Now we introduce the rocket dynamic here
############################################################

p4n=np.arange(1,20,0.25)                                                                     # assume 1-11 bar
md4n=A_throat*p4n/(np.sqrt(T_4*R/(M*gamma)))*np.power((gamma+1)/2,-(gamma+1)/(2*(gamma-1)))

# Nozzle exit
pe=p4n/np.power(1+((gamma-1)/2*Me**2), gamma/(gamma-1))                         #exit pressure (use the assumption of p_4/p_ambient>1.8)
ve=np.sqrt((2*gamma*R*T_4)/(gamma-1)/M*(1-np.power(pe/p4n,(gamma-1)/gamma)))    #exit velocity

acceleration=md4n*ve  # assume mass=1kg  -> linear relation 1

## Visualization 
plt.figure(figsize=(24,5))
plt.subplot(131)
plt.plot(p4n[:-1], pe[:-1], label='p')
plt.xlabel('nozzle inlet pressure [Bar]')
plt.ylabel('nozzle exit pressure [Bar]')
plt.grid()


plt.subplot(132)
plt.plot(p4n[:-1], ve[:-1])
plt.xlabel('nozzle inlet pressure [Bar]')
plt.ylabel('exhaust speed [m/s] ')
plt.grid()

plt.subplot(133)
plt.plot(p4n[:-1], acceleration[:-1])
plt.xlabel('nozzle inlet pressure [Bar]')
plt.ylabel('acceleration [m/s^2] ')
plt.grid()
plt.show()

In [ ]:
print(md4n)

In [ ]:
print(ve)

In [ ]:
############################################################
'''
Part 3: RK4 Validation with rocket dynamic + MPC controller control
'''

############################################################

from mpccontroller import MPCController



# Parameter Setup
# Integration parameter 
h=1/500  # step size    need at least 1000hz (sec)
t0=0    # initial time (sec)
tf=5    # final time (sec)

time=np.linspace(t0,tf,int((tf-t0)/h)+1)

###########################################################

# Motion (dynamic system)
x_t0=0.0
v_t0=0.0
a_t0=0.0

x_target=5.0         # height target [m]

R_gas=8314            # universal gas constant [J/kmol*K]
p3_t0=1e5           # inlet of the nozzle:  assume initial constant air pressure


# initialize the state at each time stamp
state_t0=np.array([p3_t0,0.0])
state_t=np.zeros((len(state_t0),len(time)))


motion_t0=np.array([x_t0,v_t0])
motion_t=np.zeros((len(motion_t0),len(time)))


state_ti=state_t0
motion_ti=motion_t0
nozzle_st=[p3_t0,0]


## Nozzle
Th_d=9e-3                        # throat diameter [m]
Th_exit_d=11e-3                  # exit diameter [m]
A_throat=np.pi*(Th_d/2)**2      #[m^2]
A_e=np.pi*(Th_exit_d/2)**2      #[m^2]
epsilon=A_e/A_throat           #expansion ratio
T_4=273                        #[k]

## constant
R_gas=8314            # universal gas constant [J/kmol*K]
M=28.013           # nitrogen gas  [kg/kmol]
gamma=1.4          # ratio of specific heats (diatomic gas)
g=9.81             # [m/s^2]

## Mach number
Me=1.8486328125000004        # from the main file, no longer do iteration


## mass
m_hop=3.5    # mass of hopper [kg]
m_hose=1     # mass of hose   [kg]
prev_pose=x_t0  # for hose dynamic modeling
k_hose=6     # variable hopper mass [N/m]
F_RR=10      # rolling resistence [N]

In [ ]:
######################################################################
## For MPC parameter
Q=np.array([[20.0,0.0],[0.0,5.0]])
R=np.array([0.1])
Qf=10000*Q
N=200

# initial input
u0=np.array([1e5]).reshape(-1,1)
z0=np.array([0.0,p3_t0]).reshape(-1,1)

#target state
xs=np.array([x_target,0.0]).reshape(-1,1)


mpc=MPCController(motion_t0,u0,z0,Q,R,Qf,xs,20,N)
mpc.setup()

In [ ]:


###########################################################
# some constants for stateupdate function (de laval nozzle)

pe_term=1/np.power(1+((gamma-1)/2*Me**2), gamma/(gamma-1)) 
print("pe_term:", pe_term)

md4n_term=A_throat / np.sqrt(T_4*R_gas/(M*gamma)) * np.power((gamma+1)/2, -(gamma+1)/(2*(gamma-1)))
print("md4n_term:", md4n_term)

ve_term=(2*gamma*R_gas*T_4)/(gamma-1)/M
print("ve_term:", ve_term)


ve_suggest_val=float(np.sqrt(ve_term * (1 - np.power(0.1615, (gamma - 1) / gamma))))
print("ve_suggest_value:", ve_suggest_val)



###########################################################
# define the RK4 (numerical integrator)

# Butcher Array
RK4matrix=np.array([
      [0,0,0,0],
      [1/4,1/4,0,0],
      [27/40,-189/800,729/800,0],       
      [1,214/891,1/33,650/891],       
      [214/891,1/33,650/891,0],      
   ])


def rk4_ex(f,t,x,h,*args):  #*args allows unknown number of arguments
   k1=f(t+h*RK4matrix[0][0],x+h*RK4matrix[0][1],*args)
   k2=f(t+h*RK4matrix[1][0],x+h*k1*RK4matrix[1][1],*args)
   k3=f(t+h*RK4matrix[2][0],x+h*k2*RK4matrix[2][1],*args)
   k4=f(t+h*RK4matrix[3][0],x+h*k3*RK4matrix[3][1],*args)

   xnext=x+h*(k1*RK4matrix[4][0]+k2*RK4matrix[4][1]+k3*RK4matrix[4][2]+k4*RK4matrix[4][3])
   return xnext

###########################################################
## ODE functions

def dx(t,motion,dotm,ve,*args):
   xdot=motion[1]
   xdotdot=dotm*ve/m_hop-g-F_RR/m_hop*np.sign(xdot)-k_hose/m_hose*xdot*h   #-k_hose/m_hose*xdot*h   # the last term (hose) assumes that the hose become the damping term

   #print("a: ",xdotdot)
   return np.array([xdot,xdotdot])



###########################################################
# main update function

def stateupdate(t,h,x,motion,nozzle_state):

   # states
   p3=x[0]
   md4=x[1]

   # motions
   x=motion[0]
   v=motion[1]

   pe=float(nozzle_state[0])
   ve=float(nozzle_state[1])

   alg_state=np.array([md4,pe])

   # control input  (introduce the time delay modify once imply the controller)

   #####################################################
   # switch to PID if needed: PID(x,h,kp,ki,kd)*1e5
   #####################################################

   # intorduce the actuator delay
   

   '''
   if p3_u>p3:                 # opening lag
      deltat=Open_t*(p3_u/1100000)
   elif p3_u<p3:               # closing lag
      deltat=Close_t*(1-p3_u/1100000)
   else:                       # same input (not closing/opening)
      deltat=h
   
   p3_actual=p3_u                                                               # discard the differential equation-> use time lag modelling
   

   '''

   p3_actual=mpc.compute_action(x0=np.array(motion).reshape(-1,1),z0=np.array(alg_state).reshape(-1,1),xs=np.array([x_target,0.0]).reshape(-1,1),u0=np.array([p3]).reshape(-1,1)) 
   #mpc.compute_action(x0=np.array(motion).reshape(-1,1),z0=np.array(alg_state).reshape(-1,1),xs=np.array([x_target,0.0]).reshape(-1,1),u0=np.array(p3)) 
   md4n=float(p3_actual*md4n_term)  # m_choked
   # Nozzle exit
   pe=p3_actual*pe_term                       #exit pressure
   ve=float(np.sqrt(ve_term*(1-np.power(pe/p3_actual, (gamma-1)/gamma))))    #exit velocity
   
   motion_n=rk4_ex(dx,t,motion,h,md4n,ve)
   #if motion_n[0]<0:
   #   motion_n[0]=0

   print(
    "dotm:",md4n,
    "ve:", ve,
    "p3_actual:", p3_actual,
    "x: ", motion_n[0], 
   )
      


   return [p3_actual,md4n],motion_n,[pe,ve]

In [ ]:
# main iteration loop

for ti in range(len(time)-1):
    #
    t=time[ti]

    #store the step
    state_t[:,ti]=state_ti
    motion_t[:,ti]=motion_ti

    # calculte the new value (RK4/Algebraic update)
    state_ti,motion_ti,nozzle_st=stateupdate(t,h,state_ti,motion_ti,nozzle_st)


    #print("p1 ",state_ti[0]," p2: ",state_ti[2]," p3: ",state_ti[4])
    #print("md1 ",state_ti[1]," md2: ",state_ti[3]," md3: ",state_ti[5]," md4: ",state_ti[6])
    #print("x: ", motion_t[0,ti])
    print("time:", t)

    #print(f'Step {t} of {len(time)-1}'.format())

In [ ]:
# part 3: motion
plt.subplot(121)
plt.plot(time[:-1], motion_t[0, :-1], label='x')
plt.xlabel('Time [s]')
plt.ylabel('Height [m]')
plt.grid()


plt.subplot(122)
plt.plot(time[:-1], motion_t[1, :-1], label='v')
plt.xlabel('Time [s]')
plt.ylabel('Velocity [m/s]')
plt.grid()